In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
end

function printer(x::Any, verbose="")
    if verbose != ""
        println(verbose)
    end
    show(stdout, "text/plain", x)
    println("")
end

using Plots, Random, Distributions, LinearAlgebra, JuliaProbo
gr();

In [ ]:
xlim = [-5.0, 5.0]
ylim = [-5.0, 5.0]
world = World(xlim, ylim)

circling_agent = Agent(0.2, 10.0 / 180 * pi)
robots = Array{RealRobot, 1}(undef, 0)
for i in 1:100
    robot = RealRobot([0.0, 0.0, 0.0], circling_agent, nothing;
        radius=0.05, color="gray", noise_per_meter=0.0,
        bias_rate_stds=(0.0, 0.0), expected_stuck_time=60.0,
        expected_escape_time=60.0)
    push!(robots, robot)
    push!(world, robot)
end
ideal_robot = IdealRobot([0.0, 0.0, 0.0], circling_agent, nothing,
    0.05, "red")
push!(world, ideal_robot)

dt = 0.1
anim = @animate for i in 1:300
    annota = "t = $(round(dt * i, sigdigits=3))[s]"
    p = draw(world, annota)
    for i in 1:100
        obsv = observations(robots[i].sensor_, robots[i].pose_)
        v, ω = decision(circling_agent, obsv)
        state_transition(robots[i], v, ω, dt)
    end
    obsv = observations(ideal_robot.sensor_, ideal_robot.pose_)
    v, ω = decision(circling_agent, obsv)
    state_transition(ideal_robot, v, ω, dt)
end
gif(anim, "ch4_sim4.gif", fps=10)